# CompositeBloq Graph Drawing

`qualtran.drawing` includes classes for drawing a `CompositeBloq` as a directed acyclic graph. This uses the popular package `graphviz` to render the drawings. The simplest way to get started is by calling `show_bloq`.

## Basic Usage

In [ ]:
from qualtran.drawing import show_bloq

# Use some test bloqs to show drawing features
from qualtran.bloqs.for_testing import TestAtom, TestParallelCombo

Each constituent bloq is a table whose header contains the bloq name. Each row is a register. Edges represent connections between soquets. Each edge is labeled with the bitsize of the connected registers. Dangling soquets (corresponding to the enclosing bloq's registers) are drawn as plain text.

In [ ]:
show_bloq(TestAtom())

In [ ]:
show_bloq(TestParallelCombo())

In [ ]:
cbloq = TestParallelCombo().decompose_bloq()
show_bloq(cbloq)

## Advanced Usage

### `PrettyGraphDrawer` and `GraphDrawer`

These classes contain the complete functionality for translating a composite bloq into a graphviz-compatible graph.

By default, we use `PrettyGraphDrawer` which abbreviates names, hides some details of reshaping bloqs, and chooses drawing parameters to give a compact visualization of the bloq. For debugging purposes or to serve as a base class for modifying drawing parameters to your liking, consider `GraphDrawer` which relies solely on graphviz defaults.

In [ ]:
from qualtran.drawing import PrettyGraphDrawer

PrettyGraphDrawer(cbloq).get_svg()

In [ ]:
from qualtran.drawing import GraphDrawer

GraphDrawer(cbloq).get_svg()

### `ClassicalSimGraphDrawer`

This simple extension of `GraphDrawer` annotates each edge according to classical data that flows through bloqs supporting the classical simulation protocol.

In [ ]:
from qualtran.drawing import ClassicalSimGraphDrawer
from qualtran.bloqs.mcmt import MultiAnd

ClassicalSimGraphDrawer(
    bloq=MultiAnd((1,1,1,1)).decompose_bloq(), 
    vals=dict(ctrl=[1,1,0,1])
).get_svg()

### Graphviz and `pydot`

We use `pydot` to manupulate graphviz graphs. You can get that graph directly:

In [ ]:
graph = GraphDrawer(cbloq).get_graph()
print(graph.to_string())

### Register Groups

We group left and right registers with shared names. This section shows some usual and unusual register specifications to test the graphviz logic for making the table rows match up.

In [ ]:
from qualtran import Bloq, Register, Signature, Side, QAny

class SignatureBloq(Bloq):
    """Placeholder bloq that lets you specify its signature."""
    def __init__(self, signature: Signature):
        self._signature = signature
        
    @property
    def signature(self) -> Signature:
        return self._signature
        
    def short_name(self):
        return 'Bloq'

In [ ]:
bloq = SignatureBloq(Signature([
    Register('x', QAny(100)),
    Register('y', QAny(200)),
]))
show_bloq(bloq)

In [ ]:
bloq = SignatureBloq(Signature([
    Register('x', QAny(100)),
    Register('a', QAny(2), shape=(2,), side=Side.LEFT),
    Register('y', QAny(200)),
]))
show_bloq(bloq)

In [ ]:
bloq = SignatureBloq(Signature([
    Register('x', QAny(100)),
    Register('a', QAny(2), shape=(2,2), side=Side.LEFT),
    Register('a', QAny(8),  side=Side.RIGHT),
    Register('y', QAny(200)),
]))
show_bloq(bloq)

In [ ]:
bloq = SignatureBloq(Signature([
    Register('x', QAny(100)),
    Register('a', QAny(2), shape=(2,2), side=Side.LEFT),
    Register('a', QAny(4), shape=(2,), side=Side.RIGHT),
    Register('y', QAny(200)),
]))
show_bloq(bloq)